In [67]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

In [68]:
infestation_history = pd.read_excel(r'data_raw/ML_BDR_20201019.xlsx')

In [69]:
infestation_history.head()

,LK,LK-Nr,LK-Rev,REVUFBADR,Jahr,ZR,Eigentumsgruppe,Zugang,Abgang
0,BZ,25,1,2501,2007,06 Juni,SW,5.0,0.0
1,BZ,25,1,2501,2007,08 August,SW,12.0,12.0
2,BZ,25,1,2501,2007,10 Oktober-Dezember,SW,2.0,0.0
3,BZ,25,1,2501,2008,04 April,SW,1.0,0.0
4,BZ,25,1,2501,2008,06 Juni,SW,2.0,0.0


## Forestry Districts

The 'REVUFBADR' column contains a unique identifier for the forstry districts. The first two digits indicate the county (Landkreis) and the last two digits indicate the number of the district in this county. 

In some forestry districts the district number (last two digits) begins with a leading 9 instead of a leading 0:

In [70]:
# display all forestry district numbers
infestation_history.REVUFBADR.unique()

array([2501, 2502, 2503, 2504, 2505, 2506, 2507, 2508, 2509, 2510, 1101,
       1201, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2191, 2192, 2193,
       2194, 2195, 2196, 2197, 2198, 2201, 2202, 2203, 2204, 2601, 2602,
       2603, 2604, 2605, 2606, 2691, 2901, 2902, 2701, 2702, 2703, 2704,
       2791, 2792, 2793, 2801, 2802, 2803, 2804, 2805, 3001, 3002, 3003,
       2301, 2302, 2303, 2304, 2305, 2306, 2401, 2402], dtype=int64)

During the observation timeframe, some of the districts (*Erzgebirgskreis* and *Meißen*) underwent a restructuring process. A leading 9 instead of a leading 0 signifies that the border of the district was different than it is today.  We can see when these changes happened with the following code:

In [71]:
infestation_history[infestation_history['LK-Rev'] >= 90].groupby('REVUFBADR').max()

,LK,LK-Nr,LK-Rev,Jahr,ZR,Eigentumsgruppe,Zugang,Abgang
REVUFBADR,,,,,,,,
2191,ERZ,21,91,2014,10 Oktober-Dezember,SW,4430.82,4701.61
2192,ERZ,21,92,2014,10 Oktober-Dezember,SW,2029.06,2185.31
2193,ERZ,21,93,2014,10 Oktober-Dezember,SW,231.00,238.00
2194,ERZ,21,94,2014,10 Oktober-Dezember,SW,445.00,460.00
2195,ERZ,21,95,2014,10 Oktober-Dezember,SW,1238.38,1219.88
2196,ERZ,21,96,2014,10 Oktober-Dezember,SW,1140.88,1157.92
2197,ERZ,21,97,2014,10 Oktober-Dezember,SW,1035.00,1268.00
2198,ERZ,21,98,2014,10 Oktober-Dezember,SW,175.75,164.35
2691,GR,26,91,2020,10 Oktober-Dezember,NSW,18000.00,15200.00


We could do this grouping by 'LK-Rev' because the two parts of 'REVUFBADR' also appear in the 'LK-Nr' and 'LK-Rev' columns seperately. This also means that they are redundant. We check if the information the three columns contain are really the same for every observation and then drop 'LK-Nr' and 'LK-Rev':

In [72]:
# first column as a string
lk_nr = infestation_history['LK-Nr'].astype(str) 
# second column as a string with leading zero
lk_rev = infestation_history['LK-Rev'].astype(str).apply(lambda x: x.zfill(2)) 

# concatenate these strings and check if they are identical to the 'REVUFBADR' column at every observation
(lk_nr + lk_rev == infestation_history['REVUFBADR'].astype(str)).all() 

True

In [73]:
# drop 'LK-Nr' and 'LK-Rev' columns because the information is also found in 'REVUFBADR'
infestation_history.drop(['LK-Nr', 'LK-Rev'], axis=1, inplace=True)

# the 'LK' column is also redundant as it contains a string that matches its 'LK-Rev' column
# since we can use it in the EDA more intuitively than just the different 'REVUFBADR' numbers we will keep it for now

Now we continue examining the cases with leading 9s. The *Stadtwald Zittau* (REVUFBADR 2691) is a special case among those special cases. According to Sachsenforst the correct procedure is to just add the corresponding observations to the forestry district *Zittau* (REVUFBADR 2601).

In [74]:
# in column 'REVUFBADR' change all occurrences of 2691 to 2601
infestation_history['REVUFBADR'] = infestation_history['REVUFBADR'].replace(2691, 2601)

In [75]:
# aggregate the values by summing them together for the 'Zugang' and 'Abgang' columns if every other column value is the same
infestation_history['Zugang'] = infestation_history.groupby(['REVUFBADR', 'Jahr', 'ZR', 'Eigentumsgruppe'])['Zugang'].transform('sum')
infestation_history['Abgang'] = infestation_history.groupby(['REVUFBADR', 'Jahr', 'ZR', 'Eigentumsgruppe'])['Abgang'].transform('sum')

# Now drop the duplicated rows that were just created
infestation_history.drop_duplicates(inplace=True)

In [76]:
infestation_history.shape

(8009, 7)

For the remaining forestry districts we need to distinguish between the old borders and the new ones. Sachsenforst supplied us with two shape files, one with all current district borders and one with only the borders of the old districts that were different than they are now. We only have to change the 'REVUFBADR' numbers for the abolished districts so they match the format with the leading 9s and then merge both geodataframes.